this part of the code does the following: for a given dinov2 segmentation find a way to create masks and boundingboxes for each colour which comes up in the image

In [34]:
import pickle

import os 
import glob
import random
import pickle
from collections import Counter
import plyfile

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import numpy as np

import os.path as osp
import sys


data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
#scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
scan_id = "02b33e01-be2b-2d54-93fb-4145a709cec5" 
new_scan_id =  "fcf66d8a-622d-291c-8429-0e1109c6bb26"
frame_idx = "000006"
curr_frame_number = "000008"
new_frame_number = "000007"
out_dir = "/local/home/ekoller/R3Scan/files/Segmentation/DinoV2/color" 
patch_h= 18
patch_w = 32
patch_h_size = 30
patch_w_size = 30
th=0.0002

In [41]:
#different approach 
# Read the image
#access the saved mask as an image
frame_path = osp.join(out_dir,  scan_id, "frame-{}.jpg".format(frame_idx))
segmented_img = cv2.imread(frame_path)
print("shape of segmetned", segmented_img.shape)

img_height, img_width, _ = segmented_img.shape  # Assuming the image is in RGB format

# Initialize the new image
patch_annos = np.zeros_like(segmented_img)

# Calculate patch dimensions
patch_width = img_width // patch_w
patch_height = img_height // patch_h

# Process each patch
for h in range(0, img_height, patch_height):
    for w in range(0, img_width, patch_width):
        # Extract the patch
        patch = segmented_img[h:h+patch_height, w:w+patch_width]

        # Reshape patch to a list of colors
        reshaped_patch = patch.reshape(-1, 3)

        # Find the most common color
        reshaped_patch_tuple = [tuple(color) for color in reshaped_patch]
        value_counts = Counter(reshaped_patch_tuple)
        most_common_value = value_counts.most_common(1)[0][0]

        # Fill the patch with the most common color
        patch_annos[h:h+patch_height, w:w+patch_width] = most_common_value

# Display the new image
cv2.imshow('patch Image', patch_annos)
cv2.waitKey(0)
cv2.destroyAllWindows()

#now turn everything into grayscale such that we can use the openc2 function to get boundingboxes for the components

#look how many colours there are in the img
unique_colors = np.unique(patch_annos.reshape(-1, 3), axis=0)
print("unique colours", unique_colors.shape)
#for visualization 
output_img = segmented_img.copy()


for color in unique_colors:
    # get the mask per colour
    mask = cv2.inRange(patch_annos, color, color)
    print("Processing color:", color) 
    # display the result
    # cv2.imshow('mask',mask)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    

    # get the individual components within that mask (connedted regions) using 8 neighboudhood connectvity
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
    
    #visualize the rectangles
    for i in range(1, num_labels):  # Start from 1 to skip the background
        x, y, w, h, area = stats[i]
        edges = np.random.randint(0, 256, size=3)
        cv2.rectangle(output_img, (x, y), (x+w, y+h), [25,167,88], 2)  # Green bounding box
        
          # Extract the mask for the current component
        component_mask = (labels == i).astype(np.uint8)
        print("component mask", component_mask) 
        # display the result
        # cv2.imshow('component mask', component_mask)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        
    print("colour done")


# display the result
cv2.imshow('Bounding Boxes', output_img)
cv2.waitKey(0)
cv2.destroyAllWindows()



[ WARN:0@335890.344] global loadsave.cpp:248 findDecoder imread_('/local/home/ekoller/R3Scan/files/Segmentation/DinoV2/color/02b33e01-be2b-2d54-93fb-4145a709cec5/frame-000006.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [36]:
#take one segmented image and try to create different masks for each connencte segment 
#do this on a patch wise level since there is some noise
#since we now have indices we can just access

#access the saved mask as an image
frame_path = osp.join(out_dir,  scan_id, "frame-{}.jpg".format(frame_idx))
segmented_img = cv2.imread(frame_path)
print("shape of segmetned", segmented_img.shape)
#only rotate here because else problem with dimensions, will be corrected in segment_dino_pipe
segmented_img = cv2.rotate(segmented_img, cv2.ROTATE_90_COUNTERCLOCKWISE)
# cv2.imshow('segmented Image', segmented_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

#create a array for the patches
patch_annos = np.zeros((patch_h, patch_w,3), dtype=np.uint8)
# iterate over patches within one image
for patch_h_i in range(patch_h):
    h_start = round(patch_h_i * patch_h_size)
    h_end = round((patch_h_i+1) * patch_h_size)
    for patch_w_j in range(patch_w):
        w_start = round(patch_w_j * patch_w_size)
        w_end = round((patch_w_j+1) * patch_w_size)
        patch_size = (w_end - w_start) * (h_end - h_start)
        patch_anno = segmented_img[h_start:h_end, w_start:w_end]
        #flatten the array to get the colours
        reshaped_patch = patch_anno.reshape(-1, patch_anno.shape[2])
        colors, counts = np.unique(reshaped_patch, axis=0, return_counts=True)
        max_idx = np.argmax(counts)
        max_count = counts[max_idx]
        if(max_count > th*patch_size):
            patch_annos[patch_h_i,patch_w_j] = colors[max_idx]




shape of segmetned (540, 960, 3)


ValueError: attempt to get argmax of an empty sequence

In [35]:
# display the quantization we have created for the image
new_image = np.zeros_like(segmented_img)

for patch_h_i in range(patch_h):
    h_start = round(patch_h_i * patch_h_size)
    h_end = round((patch_h_i + 1) * patch_h_size)
    
    for patch_w_j in range(patch_w):
        w_start = round(patch_w_j * patch_w_size)
        w_end = round((patch_w_j + 1) * patch_w_size)
        
        # Fill the corresponding region in the new image with the patch color
        new_image[h_start:h_end, w_start:w_end] = patch_annos[patch_h_i, patch_w_j]

# Display the new image
cv2.imshow('New Image', new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()